In [1]:
# libraries import 
import pandas as pd
import numpy as np

from datetime import datetime

In [3]:
# data import
skud = pd.read_csv('../data/raw/SKUD.csv')
skud.head()

,Дата,Приход.1,Уход.1,Длительность общая,Длительность раб.дня без обеда,Вых/Будни,id
0,"2021-03-01 00:00:00,000","1899-12-30 08:11:00,000","1899-12-30 17:32:00,000","9,35","8,35",Будни,ОРГ1-00791
1,"2021-03-01 00:00:00,000","1899-12-30 08:25:00,000","1899-12-30 17:32:00,000","9,1166666666666671","8,1166666666666671",Будни,ОРГ1-01826
2,"2021-03-01 00:00:00,000","1899-12-30 08:31:00,000","1899-12-30 17:32:00,000","9,0166666666666657","8,0166666666666657",Будни,ОРГ1-00713
3,"2021-03-01 00:00:00,000","1899-12-30 08:31:00,000","1899-12-30 17:32:00,000","9,0166666666666657","8,0166666666666657",Будни,ОРГ1-02127
4,"2021-03-01 00:00:00,000","1899-12-30 08:19:00,000","1899-12-30 17:32:00,000","9,2166666666666668","8,2166666666666668",Будни,ОРГ1-01739


In [4]:
# date to_datetime transformation
skud['date_skud'] = pd.to_datetime(skud['Дата'].apply(lambda x: x[:10]), format='%Y-%m-%d')
# drop column
skud.drop('Дата', axis=1, inplace=True)

In [5]:
max_date = skud['date_skud'].max()
print(max_date)
min_date = skud['date_skud'].min()
print(min_date)

2021-04-14 00:00:00
2021-03-01 00:00:00


In [6]:
skud['total_time_skud'] = skud['Длительность общая'].apply(lambda x: float(x.replace(',','.')))
skud.drop('Длительность общая', axis=1, inplace=True)

skud['work_time_skud'] = skud['Длительность раб.дня без обеда'].apply(lambda x: float(x.replace(',','.')))
skud.drop('Длительность раб.дня без обеда', axis=1, inplace=True)

In [7]:
skud['is_weekend_skud'] = np.where(skud['Вых/Будни'] == 'Выходные дни', 1, 0)
skud['is_workdays_skud'] = np.where(skud['Вых/Будни'] == 'Будни', 1, 0)

In [8]:
skud['arrive'] = pd.to_datetime(skud['Приход.1'].apply(lambda x: x[11:19]), format='%H:%M:%S')
skud['left'] = pd.to_datetime(skud['Уход.1'].apply(lambda x: x[11:19]), format='%H:%M:%S')

In [9]:
for i in skud.index:
    skud['arrive'].loc[i] = datetime.combine(skud['date_skud'].loc[i].date(
    ), skud['arrive'].loc[i].time())

/var/folders/dd/p98qrkfn48g4vx43m7wc74rc0000gp/T/ipykernel_14006/3782467486.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  skud['arrive'].loc[i] = datetime.combine(skud['date_skud'].loc[i].date(


In [10]:
for i in skud.index:
    skud['left'].loc[i] = datetime.combine(skud['date_skud'].loc[i].date(
    ), skud['left'].loc[i].time())

/var/folders/dd/p98qrkfn48g4vx43m7wc74rc0000gp/T/ipykernel_14006/2850309684.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  skud['left'].loc[i] = datetime.combine(skud['date_skud'].loc[i].date(


In [11]:
skud = skud.sort_values(by=['id', 'arrive']).drop_duplicates(subset=['id', 'date_skud'], keep='first')

In [12]:
skud.drop(['Приход.1', 'Уход.1', 'Вых/Будни'], axis=1, inplace=True)

In [13]:
def days_since_last_record(dates, max_date=max_date):
    return (max_date- dates.max()).days

In [14]:
grouped_skud = skud.groupby('id', as_index=False).agg({
    'date_skud':[days_since_last_record, 'count'],
    'total_time_skud':['sum', 'mean', 'median'],
    'work_time_skud': ['sum', 'mean', 'median'],
    'is_weekend_skud': ['sum','mean'],
    'is_workdays_skud': ['sum', 'mean']
})

In [15]:
grouped_skud.columns = grouped_skud.columns.map('_'.join).str.strip('_')

In [16]:
grouped_skud['exists_in_skud'] = 1

In [18]:
grouped_skud.to_csv('../data/prepared/grouped_skud.csv', index=False)

In [ ]:
grouped_skud

In [19]:
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.feature_selection import RFE

train = pd.read_csv('data/train_dataset_train.csv')

marked_calls = pd.merge(grouped_skud, train, how='inner', on='id').drop('id', axis=1)

for col in list(marked_calls):
    marked_calls[col] = marked_calls[col].fillna(marked_calls[col].mean())

scaler = StandardScaler()
marked_calls_st = scaler.fit_transform(marked_calls)
marked_calls_st_df = pd.DataFrame(marked_calls_st, columns = marked_calls.columns)

X_train = marked_calls_st_df.drop('type', axis=1)
y_train = marked_calls['type']
svc_lin=SVC(kernel='linear')
svm_rfe_model=RFE(estimator=svc_lin)
svm_rfe_model_fit=svm_rfe_model.fit(X_train, y_train)
feat_index = pd.Series(data = svm_rfe_model_fit.ranking_, index = X_train.columns)
signi_feat_rfe = feat_index[feat_index==1].index

print('Significant features from RFE',signi_feat_rfe)

sig_cols = signi_feat_rfe.to_list()
sig_cols.append('id')

grouped_skud_sig = grouped_skud[sig_cols].copy()
grouped_skud_sig['exists_in_skud'] = 1

grouped_skud_sig.to_csv('data/prepared/grouped_skud_sig.csv', index=False)

Significant features from RFE Index(['date_skud_days_since_last_record', 'total_time_skud_median',
       'total_time_skud_min', 'work_time_skud_median', 'work_time_skud_min',
       'is_weekend_skud_sum', 'is_weekend_skud_mean', 'is_workdays_skud_mean'],
      dtype='object')
